In [2]:
import pandas as pd
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV, cross_val_score

# load corpus
corpus = pd.read_csv('corpus.csv.gz', compression='gzip')
corpus_feat = pd.read_csv('corpus_liwc_mtx.csv.gz', compression='gzip')

# filter corpus
corpus_feat['_trusted_judgments'] = corpus['_trusted_judgments']
corpus_feat = corpus_feat[corpus_feat['_trusted_judgments'] == 3]
corpus_feat = corpus_feat.reset_index()

# fix labels to binary
lb = preprocessing.LabelBinarizer(neg_label=1, pos_label=2)
target = lb.fit_transform(corpus_feat['class'].values)
c, r = target.shape
target = target.reshape(c,)

# generate features matrix
corpus_feat.drop(corpus_feat.columns[0], axis=1,inplace=True)
corpus_feat.drop(corpus_feat.columns[0], axis=1,inplace=True)
data = corpus_feat.drop('class', 1).values

## Evaluating SVM

In [3]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
grid_search = GridSearchCV(SVC(), parameters, cv=3, n_jobs=3, verbose=3, scoring='accuracy')

grid_search.fit(data, target)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] ............... C=1, kernel=linear, score=0.674342, total= 1.1min
[CV] C=1, kernel=rbf .................................................
[CV] .................. C=1, kernel=rbf, score=0.630719, total=   0.1s
[CV] C=1, kernel=rbf .................................................
[CV] .................. C=1, kernel=rbf, score=0.630719, total=   0.1s
[CV] C=1, kernel=rbf .................................................
[CV] .................. C=1, kernel=rbf, score=0.631579, total=   0.1s
[CV] C=10, kernel=linear .............................................
[CV] ............... C=1, kernel=linear, score=0.663399, total= 2.1min
[CV] C=10, kernel=linear .............................................
[CV] ............

[Parallel(n_jobs=3)]: Done  12 out of  12 | elapsed:  8.2min remaining:    0.0s
[Parallel(n_jobs=3)]: Done  12 out of  12 | elapsed:  8.2min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=3)

In [4]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.662663755459
{'C': 1, 'kernel': 'linear'}


## Evaluating Naive Bayses

In [5]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

f1 = cross_val_score(model, data, target, cv=10, scoring='f1').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(f1)
print(acc)
print(recall)

0.715309564465
0.614782608696
0.768148820327


In [6]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

f1 = cross_val_score(model, data, target, cv=10, scoring='f1').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(f1)
print(acc)
print(recall)

0.583017300877
0.565072463768
0.592861464005
